<a href="https://colab.research.google.com/github/datastax/genai-cookbook/blob/main/raffle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Raffle ticket writer/picker with DataStax Astra DB vector database and RAGStack



This notebook takes you through the steps to build a simple application GenAI application to store (and draw) raffle winners. The idea is to be able to enter the names of conference attendees one-at-a-time. When the raffle is over, a random phrase must be entered to generate the winning embedding. The results are returned in the order of their similarity to the random phrase.

Requirements:
 - Access to the [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) 384 dimensional sentence transformer on [Hugging Face](https://huggingface.co/).
 - A free account and vector database with [Astra DB](https://astra.datastax.com/).
     - Sufficient resources for this notebook to build a collection named `raffle_data` with a:
         - 384 dimensional vector structure.
         - cosine metric.
 - An API endpoint for Astra DB.
 - An access token for Astra DB.

*Note: This notebook will create the `raffle_data` collection if it does not exist.*

## Install the DataStax RAGStack and SentenceTransformers libraries.

In [ ]:
!pip install ragstack-ai sentence-transformers

## Library Imports

In [ ]:
import json
import os

from getpass import getpass
from astrapy.db import AstraDB
from sentence_transformers import SentenceTransformer

## Environment Variables

In [ ]:
ASTRA_DB_TOKEN = getpass('Your Astra DB Token ("AstraCS:..."): ')

In [ ]:
ASTRA_DB_ENDPOINT = input('Your Astra DB API endpoint: ')
ASTRA_DB_NAMESPACE='default_keyspace'
ASTRA_DB_COLLECTION_NAME='raffle_data'

## Connect to Astra DB

In [ ]:
db = AstraDB(
    token=ASTRA_DB_TOKEN,
    api_endpoint=ASTRA_DB_ENDPOINT,
    namespace=ASTRA_DB_NAMESPACE,
)
collection = db.create_collection(ASTRA_DB_COLLECTION_NAME, dimension=384, metric="cosine")

## Initialize Sentence Transformer "all-MiniLM-L6-v2" model locally

In [ ]:
# initialize the all-MiniLM-L6-v2 model locally
model = SentenceTransformer('all-MiniLM-L6-v2')

## Enter attendee's name
A vector embedding will be generated for the attendee's name, and it will be stored in Astra DB.

In [ ]:
name = input("Enter attendee's name: ")

vector_embedding = model.encode(name)
strJson = (f'{{"_id":"{name.replace(" ","")}","name":"{name}","$vector":{str(vector_embedding.tolist())}}}')
doc = json.loads(strJson)

collection.insert_one(doc)

# show vector embedding
print(vector_embedding)

## Draw winning names

Generate a vector embedding

In [ ]:
search_string = input("Enter a phrase to generate an embedding: ")

In [ ]:
winner_embedding = model.encode(search_string)
results = collection.vector_find(winner_embedding.tolist(), limit=9, include_similarity=1, fields={"name"})
print(str(results).replace("}, {","},\n{"))